# Import Important Libraries

In [0]:
import numpy
import math
import pandas as pd
from sklearn.svm import SVC
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.externals import joblib 
import pickle

# Read Heart Dataset

In [0]:
data = pd.read_csv("heart.csv")
y = data.iloc[:, 13]
data = data.iloc[:, 0:13]

# Attributes Not Selected

In [0]:
colname=list()
colname.append("age")
colname.append("cp")
colname.append("exang")
colname.append("oldpeak")
colname.append("slope")
colname.append("ca")
colname.append("thal")
colname.append("chol")
colname.append("restecg")
for name in colname:
    data.drop([name],axis='columns',inplace=True)

# Normalization of DataSet

In [0]:
x = data.values
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(training_set)
x1_scaled = min_max_scaler.fit_transform(x)
joblib.dump(x1_scaled, 'mix_max_scaler.pkl') 
data = pd.DataFrame(x1_scaled)
data.columns = ['sex','trestbps','fbs','restecg','thalach']

# Splitting of Dataset into Training and Testing

In [0]:
train_x,test_x,train_y,test_y = train_test_split(data.copy(),y.copy(), test_size=0.25,random_state=50)

# Random Forrest Classifier

In [0]:
rf_scores = []
e = [11,21,31,41,51,61,71,81,91,101,111,121,131,141,151,161,171,181,191,201]
for i in range(len(e)):
    rf_classifier  = RandomForestClassifier(n_estimators=e[i])
    rf_classifier.fit(train_x.copy(), train_y.copy())
    rf_scores.append(rf_classifier.score(test_x.copy(), test_y.copy()))
print(rf_scores)

In [0]:
rf_classifier  = RandomForestClassifier(n_estimators=40)
rf_classifier.fit(train_x.copy(), train_y.copy())
print(rf_classifier.score(test_x.copy(), test_y.copy()))

# Saving Model

In [0]:
# save the model to disk
filename = 'finalized_model_RF.sav'
pickle.dump(rf_classifier, open(filename, 'wb'))

# SVM with GA Functions

In [0]:
def compute_SVM(c_value,gamma_value):
    svc_classifier = SVC(kernel = "rbf",random_state=0, gamma=gamma_value, C=c_value)
    svc_classifier.fit(train_x.copy(), train_y.copy())
    return svc_classifier.score(test_x.copy(), test_y.copy())

In [0]:
def cal_fitness(C,population):
    fitness_ls=list()
    result=list()
    for row in population:
        exp=0
        gamma=0
        for x in numpy.nditer(row):
            gamma+=(pow(2, exp))*x
            exp+=1
        gamma=math.sqrt(gamma)
        score=0
        if gamma > 0:
            score=compute_SVM(C,gamma)
        fitness_ls.append(score)
        new_ls=[score,C,gamma]
        result.append(new_ls.copy())
    fitness=numpy.array(fitness_ls)
    return fitness,result

In [0]:
def select_parents(population_gamma, fitness, num_parents):
    parents = numpy.empty((num_parents, population_gamma.shape[1]))
    for parent_num in range(num_parents):
        max_fitness = numpy.where(fitness == numpy.max(fitness))
        max_fitness = max_fitness[0][0]
        parents[parent_num, :] = population_gamma[max_fitness, :]
        fitness[max_fitness] = -1
    return parents

In [0]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    for k in range(offspring_size[0]):
        crossover_point = numpy.random.randint(1,parents.shape[1])
        parent1 = k%parents.shape[0]
        parent2 = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2, crossover_point:]
    return offspring

In [0]:
def mutation(offspring_crossover):
    for idx in range(offspring_crossover.shape[0]):
        values=numpy.random.randint(low=1, high=5)
        for v in range(values):
            offspring_crossover[idx,numpy.random.randint(low=0, high=10)] = numpy.random.randint(0,2)
    return offspring_crossover

In [0]:
def max_value(result,fitness):
    max_result=result[0][0]
    max_index=0
    i=0
    for row in result:
        if max_result<row[0]:
            max_result=row[0]
            max_index=i
        i+=1
    return result[max_index]

# Main Method of SVM with GA

In [0]:
chromosome_length = 10
initial_population_size = 50
parents_crossover = 10
mutation_rate_gamma = 5
mutation_rate_C = 5
population_size = (initial_population_size,chromosome_length)
new_population_C = numpy.random.randint(low=0, high=2, size=population_size)
new_population_gamma = numpy.random.randint(low=0, high=2, size=population_size)
num_generations=10
max_result_SVM=[0,0,0]
#print("")
for generation in range(num_generations):
    fitness_C=list()
    print("Generation : ", generation)
    print("")
    generation_C=1
    for row in new_population_C:
        print("Generation_C : ", generation_C)
        print("")
        generation_C+=1
        num_generations_C = 10
        exp=0
        C=0
        for x in numpy.nditer(row):
            C+=(pow(2, exp))*x
            exp+=1
        max_result=list()
        if C > 0 :
            for generation_gamma in range(num_generations_C):
                print("Generation_gamma : ", generation_gamma)
                print("")
                fitness,result = cal_fitness(C,new_population_gamma)
                parents = select_parents(new_population_gamma, fitness,parents_crossover)
                offspring_crossover = crossover(parents,offspring_size=(initial_population_size-parents.shape[0], chromosome_length))
                if (generation_gamma%mutation_rate_gamma)==0:
                    offspring_crossover = mutation(offspring_crossover)
                    parents = mutation(parents)
                new_population_gamma[0:parents.shape[0], :] = parents
                new_population_gamma[parents.shape[0]:, :] = offspring_crossover
                max_result=max_value(result,fitness)
                if max_result_SVM[0] < max_result[0]:
                    max_result_SVM[0] = max_result[0]
                    max_result_SVM[1] = max_result[1]
                    max_result_SVM[2] = max_result[2]
                print("Result: ",max_result[0],"C: ",max_result[1],"Gamma: ",max_result[2])
                print("")
            fitness_C.append(max_result[0])
        else:
            fitness_C.append(0)
    fitness=numpy.array(fitness_C)
    parents_C = select_parents(new_population_C, fitness_C,parents_crossover)
    offspring_crossover_C = crossover(parents,offspring_size=(initial_population_size-parents_C.shape[0], chromosome_length))
    if (generation_C%mutation_rate_C)==0:
        offspring_crossover_C = mutation(offspring_crossover_C)
    new_population_C[0:parents.shape[0], :] = parents_C
    new_population_C[parents.shape[0]:, :] = offspring_crossover_C
print("Optimized Result: ",max_result_SVM[0],"Optimized C: ",max_result_SVM[1],"Optimized Gamma: ",max_result_SVM[2])